In [1]:
import gym
import warnings
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

# Suppress DeprecationWarnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow as tf

# List all GPUs available to TensorFlow
gpus = tf.config.list_physical_devices('GPU')

# Check if any GPUs are available
if len(gpus) > 0:
    print("GPU is available")
    for gpu in gpus:
        print(f"GPU device: {gpu}")
else:
    print("GPU is not available")

GPU is available
GPU device: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [2]:
# Question 1: Cartpole-v1
env = gym.make(
    "CartPole-v0",
    # render_mode="human"
    )

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)


def CartPole_RL(model, discount_factor=0.95, num_choices=2):
    
    # Save rewards for visualization later
    episode_rewards = []

    for episode in range(1000):
        # print(f"CartPole-v0, episode {episode}")
        # Initiate one episode
        observation, info = env.reset()

        obs_history = []
        reward_history = []
        action_history = []

        terminated = False
        truncated = False



        # Roll out one episode
        while (not terminated) and (not truncated):

            # Make a prediction based on the state
            probabilities = model.predict(np.array([observation]), verbose=0)[0]
            # print(probabilities)

            # This line stochastically samples an action based on the probabilities
            action = np.random.choice(num_choices, p=probabilities)
            # print(action)

            obs_history.append(observation)
            action_history.append(action)

            observation, reward, terminated, truncated, info = env.step(action)

            reward_history.append(reward)
            


        
        # Discount rewards
        discounted_rewards = []
        cumulative_reward = 0
        for reward in reversed(reward_history):
            cumulative_reward = reward + discount_factor * cumulative_reward
            discounted_rewards.insert(0, cumulative_reward)

        # print(f"Cumulative Reward History: {discounted_rewards}")


        # print(f"Frame: {count_frame}")
        # print(f"Obs History: {obs_history}")
        # print(f"Reward History: {reward_history}")
        # print(f"Action History: {action_history}")



        def adjust_weights(obs_history, action_history, discounted_rewards):
            
            with tf.GradientTape() as tape:
                # Predict action probabilities
                action_probabilities = model(np.array(obs_history))
                # print(action_probabilities)

                # Get probabilities of actions that were taken
                indices = list(zip(range(len(action_history)), action_history))
                # print(indices)
                chosen_action_probs = tf.gather_nd(action_probabilities, indices)
                # print(chosen_action_probs)

                # Compute loss
                loss = -tf.math.log(chosen_action_probs) * discounted_rewards
                loss = tf.reduce_mean(loss)

                
            # Calculate gradients
            grads = tape.gradient(loss, model.trainable_variables)
            # print(grads)
            
            # Apply gradients to model weights
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

            # # Print updated model weights
            # for var in model.trainable_variables:
            #     print(var.name, var.numpy()[1])


        adjust_weights(obs_history=obs_history, action_history=action_history, discounted_rewards=discounted_rewards)


        # Append episode rewards for plotting
        total_reward = sum(reward_history)
        episode_rewards.append(total_reward)
        print(f"CartPole-v0 episode {episode}, reward sum: {total_reward}")

        clear_output(wait=True)
        
    env.close()

    # After the for loop
    plt.plot(episode_rewards)
    plt.xlabel("Episode")
    plt.ylabel("Total Reward")
    plt.show()

c:\Users\nuke2\Desktop\NW Work\Fall_02 Work\MSIA-FQ2\.venv\lib\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(


In [3]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation="relu", input_shape=(4,)),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(2, activation="softmax")
])

CartPole_RL(model)

KeyboardInterrupt: 